**https://huggingface.co/datasets/griffin/election-synthetic**


** الانتخابات الامريكيه 2024 **


**LLaMA 3.1 is December 2023**

# 1️⃣ Install & Upgrade Unsloth

In [1]:
%%capture
!pip install unsloth
# Also get the latest nightly Unsloth!
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

# 2️⃣ Load Base Model & Tokenizer

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.4: Fast Llama patching. Transformers: 4.55.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


<string>:37: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [3]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")

model.eval()
messages = [
    {"role": "user", "content": "who is partnering with Kamala Harris on the Democratic ticket In the 2024 presidential race?"}
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
).to("cuda" if torch.cuda.is_available() else "cpu")

with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=64,
        temperature=1.5,
        min_p=0.1,
        use_cache=True,
    )

print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


system

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

user

who is partnering with Kamala Harris on the Democratic ticket In the 2024 presidential race?assistant

I'm not aware of the latest information, my knowledge might be outdated. To provide a more accurate answer, I'd like to check some of the most recent news. Based on my current knowledge as of my cut-off date (December 2023)  in 2024, Joe Biden has not announced he is


# 3️⃣ Apply LoRA Fine-Tuning Setup

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.8.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# 4️⃣ Set Chat Template

```
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Hello!<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Hey there! How are you?<|eot_id|><|start_header_id|>user<|end_header_id|>

I'm great thanks!<|eot_id|>
```

In [5]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)

# 5️⃣ Format Dataset Prompts

In [ ]:
def formatting_prompts_func(examples):
    texts = []
    convos = []  # Create a list to store conversations

    for question, answer in zip(examples['question'], examples['answer']):
        # Create a conversation-like structure
        convo = [
            {'role': 'user', 'content': question},
            {'role': 'assistant', 'content': answer}
        ]
        convos.append(convo)  # Add convo to the list

        # Apply the chat template
        formatted_text = tokenizer.apply_chat_template(
            convo, tokenize=False, add_generation_prompt=False
        )
        texts.append(formatted_text)

    return {'text': texts, 'conversations': convos}  # Return the list of convos

# 6️⃣ Load & Map Dataset

In [7]:
from datasets import load_dataset
dataset = load_dataset("griffin/election-synthetic", split = "train")
print(dataset[:5])

README.md:   0%|          | 0.00/459 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/162k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/849 [00:00<?, ? examples/s]

{'question': ['Who did Kamala Harris select to be her vice presidential candidate in her campaign against Donald Trump?', 'True or False?\n\nAll governors are potential vice presidential candidates.', 'Who is leading the Democratic ticket in the 2024 presidential race?', 'If Kamala Harris and Tim Walz are standing 5 meters apart and a photographer wants to take their picture, what is the minimum focal length needed for a full-body shot if the camera sensor is 36mm wide and they occupy 80% of the frame width?', 'For which White House contender is the current leader of the Gopher State serving as the prospective vice president?'], 'answer': ["Tim Walz, Minnesota's Governor", 'False', 'Democratic presidential nominee', '4000mm', 'Kamala Harris'], 'meta': ['{"entity": "Tim Walz, Governor of Minnesota", "hops": 0}', '{"claim": "All governors are potential vice presidential candidates."}', '{"entity": "Presidential nominee", "hops": 0}', '{"reasoning_type": "physical"}', '{"hops": 4, "multi_

In [8]:
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map:   0%|          | 0/849 [00:00<?, ? examples/s]

In [9]:
dataset[5]["conversations"]

[{'content': 'Should Tim Walz be morally obligated to resign as governor of Minnesota upon accepting the vice presidential nomination?',
  'role': 'user'},
 {'content': "While there's no legal requirement, there is a strong moral argument that Tim Walz should resign as governor of Minnesota upon accepting the vice presidential nomination. This is because:\n\n1. Full commitment: The roles of governor and vice presidential candidate both demand full-time attention. Resigning would allow Walz to fully commit to the campaign without shortchanging his gubernatorial duties.\n\n2. Avoiding conflicts of interest: As a sitting governor, Walz might face situations where his decisions could be perceived as motivated by campaign interests rather than the best interests of Minnesota.\n\n3. Fair representation for Minnesota: The state deserves a full-time governor focused solely on state issues, not divided attention.\n\n4. Transparency and integrity: Resigning would demonstrate a commitment to ethi

In [10]:
dataset[5]["text"]

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nShould Tim Walz be morally obligated to resign as governor of Minnesota upon accepting the vice presidential nomination?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nWhile there's no legal requirement, there is a strong moral argument that Tim Walz should resign as governor of Minnesota upon accepting the vice presidential nomination. This is because:\n\n1. Full commitment: The roles of governor and vice presidential candidate both demand full-time attention. Resigning would allow Walz to fully commit to the campaign without shortchanging his gubernatorial duties.\n\n2. Avoiding conflicts of interest: As a sitting governor, Walz might face situations where his decisions could be perceived as motivated by campaign interests rather than the best interests of Minnesota.\n\n3. Fair representatio

# 7️⃣ Configure Trainer

In [11]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/849 [00:00<?, ? examples/s]

# 8️⃣ Train on Responses Only

In [12]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
    response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
)

Map (num_proc=2):   0%|          | 0/849 [00:00<?, ? examples/s]

In [13]:
tokenizer.decode(trainer.train_dataset[5]["input_ids"])

"<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nShould Tim Walz be morally obligated to resign as governor of Minnesota upon accepting the vice presidential nomination?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nWhile there's no legal requirement, there is a strong moral argument that Tim Walz should resign as governor of Minnesota upon accepting the vice presidential nomination. This is because:\n\n1. Full commitment: The roles of governor and vice presidential candidate both demand full-time attention. Resigning would allow Walz to fully commit to the campaign without shortchanging his gubernatorial duties.\n\n2. Avoiding conflicts of interest: As a sitting governor, Walz might face situations where his decisions could be perceived as motivated by campaign interests rather than the best interests of Minnesota.\n\n3. F

# 9️⃣ Inspect Tokenization Example

In [14]:
space = tokenizer(" ", add_special_tokens = False).input_ids[0]
tokenizer.decode([space if x == -100 else x for x in trainer.train_dataset[5]["labels"]])

"                                                        While there's no legal requirement, there is a strong moral argument that Tim Walz should resign as governor of Minnesota upon accepting the vice presidential nomination. This is because:\n\n1. Full commitment: The roles of governor and vice presidential candidate both demand full-time attention. Resigning would allow Walz to fully commit to the campaign without shortchanging his gubernatorial duties.\n\n2. Avoiding conflicts of interest: As a sitting governor, Walz might face situations where his decisions could be perceived as motivated by campaign interests rather than the best interests of Minnesota.\n\n3. Fair representation for Minnesota: The state deserves a full-time governor focused solely on state issues, not divided attention.\n\n4. Transparency and integrity: Resigning would demonstrate a commitment to ethical governance and avoid the appearance of using one office to campaign for another.\n\n5. Precedent: Many politi

# 🔟 Run Training

In [15]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 849 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Step,Training Loss
1,5.176300
2,3.586100
3,3.194700
4,3.886900
5,3.183400
6,1.881800
7,2.369000
8,2.215500
9,1.577000
10,1.217100


Unsloth: Will smartly offload gradients to save VRAM!


# 1️⃣1️⃣ Set Chat Template for Inference

In [16]:
from unsloth.chat_templates import get_chat_template

tokenizer = get_chat_template(
    tokenizer,
    chat_template = "llama-3.1",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

# 1️⃣2️⃣ First Inference Test

In [17]:
messages = [
    {"role": "user", "content": "who is partnering with Kamala Harris on the Democratic ticket In the 2024 presidential race?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

outputs = model.generate(input_ids = inputs, max_new_tokens = 64, use_cache = True,
                         temperature = 1.5, min_p = 0.1)
tokenizer.batch_decode(outputs)

['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 26 July 2024\n\n<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nwho is partnering with Kamala Harris on the Democratic ticket In the 2024 presidential race?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nTim Walz<|eot_id|>']

# 1️⃣3️⃣ Streamed Inference Output

In [18]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "who is partnering with Kamala Harris on the Democratic ticket In the 2024 presidential race?"},
]
inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

In [19]:
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 1.5, min_p = 0.1)

Tim Walz<|eot_id|>


# 1️⃣4️⃣ Save Model & Tokenizer

In [20]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/chat_template.jinja',
 'lora_model/tokenizer.json')

# 1️⃣5️⃣ Reload & Run Saved Model

In [ ]:
from unsloth import FastLanguageModel
from transformers import AutoTokenizer

# Load LoRA model for inference
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "lora_model",       # Path to your saved model
    max_seq_length = 2048,           # Match what you used in training
    dtype = None,                    # Auto-detect, or "float16"/"bfloat16"
    load_in_4bit = True               # Same as training if you used quantization
)

# Switch to inference mode for faster generation
FastLanguageModel.for_inference(model)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("lora_model")

# Example generation
messages = [
    {"role": "user", "content": "who is partnering with Kamala Harris on the Democratic ticket In the 2024 presidential race?"}
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt"
).to("cuda")

outputs = model.generate(input_ids=inputs, max_new_tokens=64, temperature=1.5, min_p=0.1)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True)[0])


==((====))==  Unsloth 2025.8.4: Fast Llama patching. Transformers: 4.55.0.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.1+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.3.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.31.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


<string>:37: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.


system

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

user

who is partnering with Kamala Harris on the Democratic ticket In the 2024 presidential race?assistant

Tim Walz
